# BETTER WAY 23. 인터페이스가 간단하면 클래스 대신 함수를 받자

### [ 참고 ]
#### defaultdict 
- defaultdict()는 말그대로 dictionary 에 기본값을 정의하고 키값이 없더라도 에러를 출력하지않고 기본값을 출력 
- dict.set_default를 사용하는것보다 빠른것이 장점
- https://dongyeopblog.wordpress.com/2016/04/08/python-defaultdict-사용하기/

#### nonlocal
- nonlocal 은 전역변수를 포함한 상위 스코프의 이름을 사용한다는 뜻이다.

#### normal vs nonlocal vs global
- normal scope 예제
- nonlocal scope 예제
- global scope 예제

In [11]:
x = 0
def outer():
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 0


In [12]:
x = 0
def outer():
    x = 1
    def inner():
        nonlocal x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 2
global: 0


In [14]:
x = 0
def outer():
    x = 1
    def inner():
        global x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 2


<br/>

### [ 본문 ]

In [1]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x:len(x), reverse=False)
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


In [38]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue':3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]
result = defaultdict(log_missing, current)
print("Before:", dict(result))
for key, amount in increments:
    result[key] += amount
print("After:", dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [39]:
result['black']
print(dict(result))

Key added
{'green': 12, 'blue': 20, 'red': 5, 'orange': 9, 'black': 0}


In [40]:
current = {'green': 12, 'blue':3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]

def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count    # 상태 보존 클로저
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

In [42]:
result, count = increment_with_report(current, increments)
assert count == 2

In [45]:
result

defaultdict(<function __main__.increment_with_report.<locals>.missing>,
            {'blue': 20, 'green': 12, 'orange': 9, 'red': 5})

In [19]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

In [20]:
counter = CountMissing()
result = defaultdict(counter.missing, current)    # 메서드 참조

for key, amount in increments:
    result[key] += amount

assert counter.added == 2

In [21]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0

In [23]:
counter = BetterCountMissing()
counter()
assert callable(counter)

In [27]:
counter = BetterCountMissing()
result = defaultdict(counter, current)    # __call__이 필요함
for key, amount in increments:
    result[key] += amount
assert counter.added == 2